In [76]:
import pandas as pd
import numpy as np

In [77]:
movies=pd.read_csv('tmdb_5000_movies.csv')
credits=pd.read_csv('tmdb_5000_credits.csv')

In [78]:
movies.head(1)

In [79]:
credits.head()

In [80]:
movies.info()

In [7]:
movies=movies.merge(credits,on='title')

In [8]:
movies.head(1)

In [9]:
movies.info()

In [10]:
#Useful Columns:-
#genres
#id
#keywords
#Overview
#title
#cast
#crew


In [11]:
# movies['original_language'].value_counts()

In [12]:
movies=movies[['title','id','overview','keywords','genres','cast','crew']]

In [13]:
movies.head(1)['cast']

In [14]:
movies.head(3)

In [15]:
movies.isnull().sum()

In [16]:
movies.dropna()

In [17]:
movies.duplicated().sum()

In [18]:
movies.iloc[0].genres

In [19]:
import ast

In [20]:
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [21]:
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

In [22]:
movies['genres']=movies['genres'].apply(convert)

In [23]:
movies.head()

In [24]:
movies.iloc[0].keywords

In [25]:
movies['keywords']=movies['keywords'].apply(convert)

In [26]:
movies.head()

In [27]:
# movies.iloc[0].cast

In [28]:
def convert3(obj):
    L=[]
    cnt=0
    for i in ast.literal_eval(obj):
        if cnt!=3:
            L.append(i['name'])
            cnt=cnt+1
        else:
            break
        
    return L

In [29]:
movies['cast']=movies['cast'].apply(convert3)

In [30]:
movies.head()

In [31]:

# for i in ast.literal_eval(movies.iloc[0].crew):
#     L1=[]
#     if i['job']=='Director':
#         L1.append(i['name'])
#     print(L1)

In [32]:
def convert4(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
        
            L.append(i['name'])
    return L

In [33]:
movies['crew']=movies['crew'].apply(convert4)

In [34]:
movies.head(5)

In [35]:
movies.info()

In [36]:
def split_string(x):
    if isinstance(x, str):
        return x.split(" ")
    else:
        return []

movies['overview'] = movies['overview'].apply(split_string)


In [37]:
movies['overview']

In [38]:
movies.head()

In [39]:
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [40]:
movies.head(5)

In [41]:
movies['tags']=movies['overview']+movies['keywords']+movies['genres']+movies['cast']+movies['crew']

In [42]:
 movies.head()

In [43]:
new_df=movies[['title','id','tags']]

In [44]:
new_df

In [45]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

In [46]:
new_df.head()

In [47]:
new_df['tags'][0]

In [48]:
#lower case
new_df['tags'].apply(lambda x: x.lower())

In [49]:
from sklearn.feature_extraction.text import CountVectorizer

In [50]:
cv=CountVectorizer(max_features=5000,stop_words='english')

In [51]:
vectors=cv.fit_transform(new_df['tags']).toarray()

In [52]:
vectors[0]

In [53]:
cv.get_feature_names_out()

In [54]:
#Now we will use steming to drop similar features
!pip install nltk
import nltk

In [55]:
from nltk.stem.porter import PorterStemmer

In [56]:
ps=PorterStemmer()

In [57]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [58]:
ps.stem("dancing")

In [59]:
stem("loving")

In [60]:
stem('In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d Action Adventure Fantasy ScienceFiction SamWorthington ZoeSaldana SigourneyWeaver JamesCameron'
)

In [61]:
new_df['tags'].apply(stem)

In [62]:
# cv.get_feature_names()

In [63]:
from sklearn.metrics.pairwise import cosine_similarity

In [64]:
cosine_similarity(vectors)

In [65]:
cosine_similarity(vectors).shape

In [66]:
similarity=cosine_similarity(vectors)

In [67]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]

In [68]:
def recommend(movie):
    movie_index=new_df[new_df['title']==movie].index[0]
    distances=similarity[movie_index]
    movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    for i in movies_list:
        print(new_df.iloc[i[0]].title)


In [74]:
new_df[new_df['title']=='Spectre'].index[0]

In [75]:
recommend('Spiderman')